# Decorator
---

## Introduction 
在進階的python程式碼中，可以透過裝飾器對函式進行裝飾，讓函式達到特定的變化。為了要讓各位邁向更高的程式設計的境界，在本節將會介紹裝飾器的使用。

不過在正式進入裝飾器的介紹前，我們要先了解幾個概念: 
1. 在python中，函式也是一個物件(參考M1:物件導向課程)，可被賦值給變數，可作為參數傳遞，也可作為其他函數的回傳值。
2. 函式裡面可以再定義函式，我們稱其為子函式，而包含此子函式的函式為父函式。

### 1. 函式內定義函式用法介紹 
--- 

定義階段: 
- Step 1: 父函式`parent_func`裡定義子函式`child_func`
- Step 2: 子函式`child_func`用了父面函式`parent_func`的變量`parent_x`和輸入變量`num1`。
- Step 3: 父函式`parent_func`輸出子函式`child_func`

In [1]:
def parent_func(num1):
    parent_x = 5
    #print('Step 1: 父函式`parent_func`裡定義子函式`child_func`')
    def child_func(num2): 
        #print('Step 2: 子函式`child_func`用了父面函式`parent_func`的變量`parent_x`和輸入變量`num1`')
        child_result = parent_x*num2 + num1
        print('child_result:', child_result)
        return child_result
    #print('Step 3: 父函式`parent_func`輸出子函式`child_func`')
    return child_func

使用階段: 
- Step 4: 執行`parent_func`帶出`child_func`
- Step 5: 執行`child_func`(f)

In [2]:
#print('Step 4: 執行`parent_func`帶出`child_func`')
f = parent_func(100)
#print('Step 5: 執行`child_func`(f)')
f(5) # 呼叫 fun2

child_result: 125


125

父函式執行結束後，即變父函數被刪除，傳入子函數的資料仍會被保留，就像被打包進了子函數。在軟體開發中，這個概念叫做閉包(closure)。

In [3]:
f = parent_func(100) # f 指向 fun2
print('before deletion of parent_func')
f(5)
del parent_func
print('after deletion of parent_func')
f(5)

before deletion of parent_func
child_result: 125
after deletion of parent_func
child_result: 125


125

透過以下方式可以對傳入閉包(`f`)中的參數進行檢視: 

In [4]:
print('num1:')
print(f.__closure__[0].cell_contents) 

print('parent_x:')
print(f.__closure__[1].cell_contents) 

num1:
100
parent_x:
5


### 2.裝飾器介紹

--- 

裝飾器可以用來修飾(或包裝)函式，可以增加額外的程式碼在被包裝的目標函式的之前或之後。

想像裝飾器是燈罩，加上燈罩之後，可以修改電燈照明範圍；你也可以改燈罩的顏色，讓他有不同顏色的輸出。

但電燈的功能本身沒有改變，只是被燈罩修改了照明範圍或顏色。

裝飾器如同上面的舉例，可以隨時加上去很方便！

裝飾器有以下優點：

- 擴充容易
- 重複利用

以下對裝飾起的使用方式進行說明: 

首先我們創建一個希望被包裝的函式，叫做`say_hello`。

In [5]:
def say_hello():
    print('inside say_hello')
    return 'Hello'

say_hello()

inside say_hello


'Hello'

透過一個父函式，和其中的`wrapper_func`，我們可以去包裝出一個新的函式`add_something`。
這個父函式可以吃進被包裝的函式，加上新程式碼，輸出包裝後的函式。

* 講師提醒: 

    - \*args : 收集所有位置型參數，以 tuple 存在 args

    - \**kwargs: 收集所有關鍵字參數，以 dict 存在 kwargs

In [6]:
def add_something(func):
    def wrapper_func(*args, **kargs):
        print('something before say_hello')
        ans = func(*args, **kargs)
        print('something after say_hello')
        return ans 
    return wrapper_func
new_func = add_something(say_hello)

In [7]:
new_func()

something before say_hello
inside say_hello
something after say_hello


'Hello'

我們可以用 python 裝飾器專用的語法糖(`@`)來改寫: 

In [8]:
@add_something
def say_hello():
    print('inside say_hello')
    return 'Hello'

In [9]:
say_hello()

something before say_hello
inside say_hello
something after say_hello


'Hello'

### 裝飾器範例: 

#### 小寫裝飾器

In [10]:
def lowercase(func):
    def wrapper_func(*args):
        print("原函式執行前")
        original_result = func(*args)
        print("original_result:", original_result)
        print("原函式執行後")
        modified_result = original_result.lower()
        print("modified_result:", modified_result)
        return modified_result
    return wrapper_func

@lowercase
def say_hello():
    return 'Hello'

@lowercase
def say_hi():
    return 'Hi'

In [11]:
say_hello()

原函式執行前
original_result: Hello
原函式執行後
modified_result: hello


'hello'

In [12]:
say_hi()

原函式執行前
original_result: Hi
原函式執行後
modified_result: hi


'hi'

#### 大寫裝飾器 

In [13]:
def uppercase(func):
    def wrapper_func(*args):
        original_result = func(*args)
        modified_result = original_result.upper()
        return modified_result
    return wrapper_func

@uppercase
def say_hello():
    return 'Hello'

@uppercase
def say_hi():
    return 'Hi'

In [14]:
say_hello()

'HELLO'

In [15]:
say_hi()

'HI'

#### 計時器裝飾器 

In [16]:
import datetime
import time 
def timer(func):
    def wrapper(*args, **kwargs):
        start = datetime.datetime.now()
        result = func(*args, **kwargs)
        end = datetime.datetime.now()
        print(f"執行時間：{ end - start } 秒")
        return result
    return wrapper

In [17]:
@timer
def sleep():
    time.sleep(1)

In [18]:
sleep()

執行時間：0:00:01.001043 秒


#### 異常處理裝飾器 

--- 

在06_M2_Error_and_Exception的章節，我們有學到可以用`try...except`在異常發生時，將異常訊息印出來。這個功能可以透過decorator來快速實現，讓每一個函式都可以在發生異常時，印出該函式對應的異常訊息，幫助我們監控程式的執行狀況。



In [19]:
def print_error_message(func):
    def wrapper(*args, **kwargs):
        try:
            result = func(*args, **kwargs)
        except:
            print(f'{func.__name__} 發生問題') 
        return result
    return wrapper

In [20]:
@print_error_message
def function_with_error():
    raise ValueError('Error') 
    

In [21]:
function_with_error() 

function_with_error 發生問題


UnboundLocalError: local variable 'result' referenced before assignment

In [22]:
@print_error_message 
def divide_by_zero():
    return 1./0.
divide_by_zero() 

divide_by_zero 發生問題


UnboundLocalError: local variable 'result' referenced before assignment

In [23]:
@print_error_message
def function_without_error():
    print('inside function without error')

In [24]:
function_without_error() 

inside function without error


#### 重複執行裝飾器 (重複n次) 

--- 

透過兩層的`wrapper`，我們可以實現具有變化性的decorator。這裡，我們希望有一個decorator是把函數重複執行n次，並且n的數值可以自由決定。

透過以下方法可以實現，我們需要一個`decorator_wrapper`，用來產生一個重複執行n次的decorator，裡面則是包一個`func_wrapper`，用來讓function重複執行。

In [25]:
import datetime
import functools

def repeat(n):
    def decorator_wrapper(func):
        def func_wrapper(*args, **kwargs):
            for _ in range(n):
                result = func(*args, **kwargs)
            return result
        return func_wrapper
    return decorator_wrapper 

@repeat(5)
def say_hello():
    print('Hello')

In [26]:
say_hello()

Hello
Hello
Hello
Hello
Hello


### 使用functools.wraps進行更穩當的裝飾

前面所撰寫的裝飾器是把一個函式置換成`warpper_func`，因此原本的函式名稱和docstring會被`warpper_func`的名稱和docstring覆蓋掉。

我們可以使用`functools.wraps`套件來避免這個問題 

使用`functools.wraps`前: 

In [27]:
def uppercase(func):
    def wrapper_func(*args):
        '''
        This is the docstring of wrapper_func. 
        '''
        original_result = func(*args)
        modified_result = original_result.upper()
        return modified_result
    return wrapper_func

@uppercase
def say_hello():
    '''
    This is the docstring of say_hello. 
    '''
    return 'Hello'
print(say_hello.__name__) 
print(say_hello.__doc__)

wrapper_func

        This is the docstring of wrapper_func. 
        


使用`functools.wraps`後，函式名稱和docstring可以保持一致

In [28]:
import functools
def uppercase(func):
    @functools.wraps(func)
    def wrapper_func(*args):
        '''
        This is the docstring of wrapper_func. 
        '''
        original_result = func(*args)
        modified_result = original_result.upper()
        return modified_result
    return wrapper_func

@uppercase
def say_hello():
    '''
    This is the docstring of say_hello. 
    '''
    return 'Hello'
print(say_hello.__name__) 
print(say_hello.__doc__)

say_hello

    This is the docstring of say_hello. 
    


### 於class中定義裝飾器
--- 
decorator也可以於class中使用 

In [29]:
class Animals:
    def __init__(self, name):
        self.name = name
        print(f"My name is {self.name}")
    def talk(self):
        pass
    def author(self):
        self.__title = 'Erik'
    def author_title(self):
        return self.__title

class Cat(Animals):
    def __init__(self, cat_name):
        super().__init__(cat_name)
        
    def uppercase(func):
        @functools.wraps(func)
        def wrapper_func(*args):
            '''
            This is the docstring of wrapper_func. 
            '''
            original_result = func(*args)
            modified_result = original_result.upper()
            return modified_result
        return wrapper_func
    @uppercase 
    def talk(self):
        return "meow meow meow"

In [30]:
cat = Cat('Tom')
cat.talk()

My name is Tom


'MEOW MEOW MEOW'